In [4]:
import sys

sys.path.append("../")

import torch.nn as nn
import torch

from abl.abducer.abducer_base import HED_Abducer
from abl.abducer.kb import HED_prolog_KB

from abl.utils.plog import logger
from abl.models.basic_model import BasicModel
from abl.models.wabl_models import WABLBasicModel

from models.nn import SymbolNet
from datasets.hed.get_hed import get_hed, split_equation
from abl import framework_hed

In [5]:
# Initialize logger
recorder = logger()

### Logic Part

In [6]:
# Initialize knowledge base and abducer
kb = HED_prolog_KB(pseudo_label_list=[1, 0, '+', '='], pl_file='./datasets/hed/learn_add.pl')
abducer = HED_Abducer(kb)

ERROR: /home/gaoeh/ABL-Package/examples/datasets/hed/learn_add.pl:67:9: Syntax error: Operator expected


### Machine Learning Part

In [ ]:
# Initialize necessary component for machine learning part
cls = SymbolNet(
    num_classes=len(kb.pseudo_label_list),
    image_size=(28, 28, 1),
)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(cls.parameters(), lr=0.001, weight_decay=1e-6)

In [ ]:
# Pretrain NN classifier
framework_hed.hed_pretrain(kb, cls, recorder)

In [ ]:
# Initialize BasicModel
# The function of BasicModel is to wrap NN models into the form of an sklearn estimator
base_model = BasicModel(
    cls,
    criterion,
    optimizer,
    device,
    save_interval=1,
    save_dir=recorder.save_dir,
    batch_size=32,
    num_epochs=1,
    recorder=recorder,
)

### Use WABL model to join two parts

In [ ]:
model = WABLBasicModel(base_model, kb.pseudo_label_list)

### Dataset

In [ ]:
total_train_data = get_hed(train=True)
train_data, val_data = split_equation(total_train_data, 3, 1)
test_data = get_hed(train=False)

### Train and save

In [ ]:
model, mapping = framework_hed.train_with_rule(model, abducer, train_data, val_data, select_num=10, min_len=5, max_len=8)
framework_hed.hed_test(model, abducer, mapping, train_data, test_data, min_len=5, max_len=8)

recorder.dump()